In [5]:
!pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

   ---------------------------------------- 0.0/14.0 MB ? eta -:--:--
    --------------------------------------- 0.3/14.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/14.0 MB 1.7 MB/s eta 0:00:08
   --- ------------------------------------ 1.3/14.0 MB 2.5 MB/s eta 0:00:06
   ----- ---------------------------------- 1.8/14.0 MB 2.8 MB/s eta 0:00:05
   -------- ------------------------------- 2.9/14.0 MB 3.0 MB/s eta 0:00:04
   ---------- ----------------------------- 3.7/14.0 MB 3.2 MB/s eta 0:00:04
   --------------- ------------------------ 5.2/14.0 MB 3.8 MB/s eta 0:00:03
   ---------------- ----------------------- 5.8/14.0 MB 3.7 MB/s eta 0:00:03
   ------------------- -------------------- 6.8/14.0 MB 3.9 MB/s eta 0:00:02
   ------------------------ --------------- 8.4/14.0 MB 4.3 MB/s eta 0:00:02
   --------------------------- ------------ 9.7/14.0 MB 4.5 MB/s eta 0:00:01
   -------------------------------- ------- 11.3/14.0 MB 4.8 MB/s eta 0:00:01
   ---------

In [227]:
!pip install markdown

In [10]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

# For sending mail
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# For rendering markdown
import markdown

# For creating UI
import gradio as gr

In [11]:
#for email authentication
from __future__ import print_function
import os.path
import base64

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

In [12]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [13]:
# Call the LLM with the input variables

In [14]:
exceptions = []

In [15]:
# Building system prompt
def get_verse_system_prompt():
    system_prompt = "You are a spiritual student who classifies the versus of the BhagavadGita according to a given theme.\n"
    system_prompt += "Given a theme, you should pick a verse from any chapter and give it's location in the form of index chapter.verse_number (3.21)\n"
    system_prompt += "You should respond in JSON as in this example:\n"
    system_prompt += """
        {"title": "Chapter 3, Verse 21 (3.21)", "verse": "कर्मणा ह्यपि संसिद्धिम्‌
                                                  आस्थिता जनकादय:।
                                                  लोकसंग्रहमेवापि
                                                  सम्पश्यन्कर्तुमर्हसि॥"}
        """
    return system_prompt

In [28]:
# Define user prompt
def get_verse_user_prompt():
    user_prompt = f'''
                    Here is the theme : {theme_input},
                    Please find a verse from BhagavadGita excluding {exceptions} for a given theme {theme_input}
                    '''#excluding those results which are already used
    
    user_prompt += "If the verse is not in the exceptions for a given theme and used for a different theme, you are free to suggest it for a different theme."
    return user_prompt

In [17]:
#Call openAI to return versus
def get_verses(theme):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": get_verse_system_prompt()},
            {"role": "user", "content": get_verse_user_prompt()}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    result = json.loads(result)

    #Remember those results which are suggested now
    combination = (theme, result['title'])
    exceptions.append(combination)
    return result

In [18]:
# So far we have fetched the new verses relevant to a given theme 
# Lets generate a script for producting youtube video

In [30]:
#def function for system prompt
def get_script_system_prompt():
    sys_prompt = 'You are a script writer for a youtube spiritual channel\n'
    sys_prompt += 'You are given a verse like below: \n'
    sys_prompt += str(get_verses(theme_input))
    sys_prompt += '\n'
    sys_prompt += f'Give me an engaging script with markdowns in a {tone_input} tone for a {format_type_input} format video for audience like youth seeking purpose, spiritual seekers, indians abroad, scholars and curious minds.'

    return sys_prompt

In [31]:
# def function for user prompt
def get_script_user_prompt():
    user_prompt = f'Given the verse, help me generate a detailed script suitable for {format_type_input} format video.\n'
    user_prompt += f'Please give me the complete verse, its meaning, a relevant story having a dilemma which the verse solves and the interpretation of the verse with respect to {theme_input}.\n'
    user_prompt += 'Let the script give cues about video editing, host actions.'
    user_prompt += 'given the below example, please follow the format:\n'
    user_prompt += f"""
    [Opening Scene - Soft Instrumental Music Playing]

    [Cut to Host in a serene setting, perhaps by a river or in a lush green garden.]

    **Host**: (Smiling at the camera) "Namaste, dear viewers! Welcome back to our channel, where we explore the depths of spirituality and seek to ignite the flame of wisdom within you. Today, we delve into a profound verse from the Bhagavad Gita that speaks to the very essence of life and identity, another verse which relates to {theme_input}"

    ---
    
    [Text On Screen: Chapter 2, Verse 13 (2.13)]

    **Host**: (With a sense of reverence) "Let’s first take a moment to recite this verse together. It goes like this:

    > देहिनोऽस्मिन्न्यथा देहे कौमारं यौवनं जरा।
    > तथादेहान्तरप्राप्तिर्धीरस्तत्र न मुह्यति॥

    Now, let’s understand the essence of this verse."

    [Cut to Graphic: Verse Translation with Key Concepts Highlighted]

    *Host Voiceover*: (Calm and engaging tone) "The meaning of this beautiful verse translates to: 'Just as the body undergoes changes from childhood to old age, similarly, the soul transitions from one body to another. The wise, who understand this, are never bewildered by these changes.'

    [Cut back to Host]

    **Host**: (Nodding, creating a connection)
    "So, why is this verse so important, especially for us as young seekers of purpose? It highlights a profound truth—that our identities are not confined by our physical forms or the stages of life we experience. Instead, we are eternal beings who are constantly evolving."

    --- 
    
    [Scene Transition - Soft Music Playing]

    [Cut to a Story Animation - A young man named Arjun in a busy city]

    *Host (Voiceover)*: "Let me share a relatable story. Meet Arjun. Like many of us, he was once full of dreams and aspirations. He excelling in school, pursuing a career in engineering. But as the years passed, he faced a crossroads. As the pressure mounted, he began to question his identity.

    (Visuals show Arjun overwhelmed by societal expectations, with people pushing him in different directions.)

    He felt distinct phases of life pulling at him: childhood dreams, youthful ambitions, and the looming responsibilities of adulthood. The changing seasons of his life left him confused and wondering if he had lost his true self."

    [Cut back to Host, empathetic tone]

    **Host**: "Have you ever felt like Arjun? It’s a dilemma we all face, especially in today's fast-paced world where expectations can cloud our true identity. But just like our verse suggests, we should recognize that these changes don’t define us. They are simply part of the journey."

    [Scene Transition - Calm Music Playing while Host meditates]

    ---
    
    **Host**: (Speaking gently) "Let’s take a moment to reflect. When we are sad, does that sadness define us? Or when we achieve success, do we become defined solely by that success? The answer isn't as straightforward as it seems. Here’s the catch: our essence is beyond these transient states. Like the body, our identities are fluid."

    [Cut to Visuals of Nature - flowing rivers, trees shedding leaves, etc.]

    *Host Voiceover*: "Imagine the endless cycle of nature—the changing seasons, the growth, the decay, and rebirth. Just like the leaves that drop to make way for new growth, our experiences contribute to our spiritual evolution."

    [Cut back to Host - Inviting and Warm Tone]

    **Host**: "Just as the wise who understand the transformation of the soul remain unshaken, we, too, can cultivate that wisdom to rise above the chaos of change. Recognize your true essence—beyond the body, the roles, the titles. Understand that your spirit is eternal."

    [Scene Transition - Soft Inspirational Music Begins]

    **Host**: (Passionately) "So how can we embody this truth in our daily lives? Here’s a small exercise: Each day, take a few moments to meditate on who you really are. Write down what aspects of your identity are tied to transient things. Challenge yourself—what happens when you peel these layers away?"

    [Cut to host with a pad, writing ideas]

    [Scene Transition - Editing Cues - Show engaging graphics of identity, layers of a person, etc.]

    *Host Voiceover*: "Each effort towards understanding and embracing our true self draws us closer to the realization that we are eternal souls, having a human experience. This is the wisdom that can empower you to stand tall against the adversities of life."

    [Cut back to Host]

    ---

    **Host**: (Concluding) "Thank you for joining me today in this exploration of Chapter 2, Verse 13 of the Bhagavad Gita. Remember, when you feel lost in the complexities of life, return to this teachings and remind yourself that you are not just a body; you are an eternal being on a magnificent journey.

    [Closing Scene - Uplifting Music Playing]

    **Host**: "Don’t forget to like, share, and subscribe if you found resonance in this message. And share your thoughts in the comments below. What did you find most challenging in your own journey of self-identity? Let’s connect and support each other in our spiritual quests. Until next time, stay enlightened, stay inspired!"

    [End Screen with Subscribe Button and Previous Video Suggestions]

    [End of Script]
    """
    
    return user_prompt

In [21]:
def create_script():
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": get_script_system_prompt()},
            {"role": "user", "content": get_script_user_prompt()}
          ],
    )
    result = response.choices[0].message.content
    return result

In [22]:
# If modifying scopes, delete the file token.json
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

def get_service():
    creds = None

    # Try loading existing token
    if os.path.exists('token.json'):
        try:
            creds = Credentials.from_authorized_user_file('token.json', SCOPES)
        except Exception:
            print("token.json is invalid, removing...")
            os.remove('token.json')
            creds = None

    # If no creds or creds invalid, do login
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # This must successfully return creds
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)

        # Save only if creds is valid
        if creds:
            with open('token.json', 'w') as token:
                token.write(creds.to_json())
        else:
            raise Exception("Failed to obtain credentials from Google login.")

    return build('gmail', 'v1', credentials=creds)


In [23]:
# Email details
def updated_body(message_body):
    body = '''Hello!\n
    
Please find below the script:\n
    
    ---\n
    '''
    body += message_body
    body +='\n---'
    body +='''\n\nThanks,\n
Vaishnavi'''
    return body

In [24]:
def send_email(body_text):
    subject = f'Script for the theme : {theme_input}'
    global to, s_message

    #setting variables for connection
    to = receiver_email_input
    service=get_service()

    # Convert Markdown to HTML
    html_content = markdown.markdown(body_text)

    #Setting mail parameters
    message = MIMEText(html_content, "html")
    message['to'] = receiver_email_input
    message['subject']=subject
    raw = base64.urlsafe_b64encode(message.as_bytes()).decode()
    message_body = {'raw':raw}
    s_message = service.users().messages().send(userId='me', body = message_body).execute() 
    return s_message['id']

In [25]:
def process_request(theme, format_type, tone, receiver_email):
    
    global theme_input, format_type_input, tone_input, receiver_email_input
    
    #store inputs
    theme_input = theme
    format_type_input = format_type
    tone_input = tone
    receiver_email_input = receiver_email
    
    #LLM call with the inputs
    body = create_script()

    content = updated_body(body)

    #Send email with LLM response
    message_id = send_email(content)
    
    return f'The script has been sent to {receiver_email_input}'
    

In [26]:
def main():
    global theme_input, format_type_input, tone_input, receiver_email_input
    with gr.Blocks() as UI:
        gr.Markdown('# Welcome to the Gita scripting assistant!')
        gr.Markdown('## How can I help you today? 🙂')
        theme_input = gr.Textbox(label='theme')
        format_type_input = gr.Dropdown(['Long','Short'], label='Please mention the format')
        tone_input = gr.Textbox(label='tone')
        receiver_email_input = gr.Textbox(label='Enter your email')
        submit_btn = gr.Button('Generate and Send email')

        output = gr.Textbox(label='Status')

        submit_btn.click(
                    fn = process_request,
                    inputs = [theme_input, format_type_input,tone_input, receiver_email_input],
                    outputs = output
        )

        UI.launch()

In [29]:
if __name__ == "__main__":
    #Initiating the global variables
    theme_input = None
    format_type_input = None
    tone_input = None
    receiver_email_input = None
    
    #Call the main function
    main()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\vaish\anaconda3\envs\llms\Lib\site-packages\gradio\queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\vaish\anaconda3\envs\llms\Lib\site-packages\gradio\route_utils.py", line 350, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\vaish\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 2235, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\vaish\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 1746, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\vaish\anaconda3\envs\llms\Lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return awai